# D-Hydro Boezemmodel V4

## Introductie

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\beheerregister

<b>let op:</b> deze notebook werkt alleen in combinatie met een config.py. In config.py dient een variabele dimr_path te zijn opgegeven, afhankelijk van de D-Hydro installatie. Bijvoorbeeld:

dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path
from config import dimr_path

beheerregister = Path(r".\beheerregister").absolute().resolve()

invoerbestanden = {"modelgebied": "Bemalings_en_lozingsgebieden.shp",
                   "branches": "Afaanvoervakken_DHYDRO_v5.shp",
                   "profielpunten": "profielpunten_3feb2021_actueel.shp",
                   "bruggen": "Bruggen_(doorstroomopeningen).shp",
                   "duikers": "duikers.shp",
                   "sifons": "sifons.shp",
                   "stuwen": "stuwen.shp",
                   "inlaten": "inlaten.shp",
                   "gemalen": "gemalen_v2.shp",
                   "peilgebieden": "Peilgebieden.shp"}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### snap_to_branches
De optie om bij het inlezen te snappen naar branches en alles dat niet op boezemmodel_v4 ligt weg te gooien. Wordt nu alleen gebruikt bij duikers

In [3]:
branches = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
branches.columns = branches.columns.str.lower()
snap_to_branches = dict(branches=branches,
                        distance=5,
                        attribute_filter={"hydromodel": "Boezemmodel_v4"}) 


### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [4]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"hydromodel": "Boezemmodel_v4"},
                           keep_columns=["HydroModel",
                                         "bodemhoogtebovenstrooms",
                                         "bodembreedte",
                                         "taludhellinglinkerzijde",
                                         "taludhellingrechterzijde",
                                         "IWS_W_WATB"],
                           column_mapping={"OVKIDENT": "code",
                                           "IWS_AVVHOB": "bodemhoogtebovenstrooms",
                                           "AVVBODDR": "bodembreedte",
                                           "AVVTALUL": "taludhellinglinkerzijde",
                                           "AVVTALUR": "taludhellingrechterzijde"}
                          )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 4

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principe profielen weg die we later in deze notebook gebruiken.

In [5]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogtebovenstrooms",
                                                  "iws_w_watb",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde")

principe_profielen_df.to_excel(r".\xlsx\principe_profielen.xlsx")

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["WL", "Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {item: [] for item in ["code",
                              "bodemhoogtebovenstrooms",
                              "bodemhoogtebenedenstrooms",
                              "bodembreedte",
                              "hoogteinsteeklinkerzijde",
                              "geometry",
                              "codegerelateerdobject"]}

for code, df in grouper:
    data["code"] += [f"PRO_{code}"]
    data["codegerelateerdobject"] += [code]
    for item in ["bodemhoogtebovenstrooms",
                 "bodemhoogtebenedenstrooms",
                 "hoogteinsteeklinkerzijde",
                 "geometry"]:
        data[item] += [df.iloc[0][item]]
    data["bodembreedte"] += [df['bodembreedte'].sum()]

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x-0.1,
                                                    x["geometry"].y-0.1],
                                                  [x["geometry"].x+0.1,
                                                   x["geometry"].y+0.1]])),
                            axis=1)

gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [8]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={"doorstro_2": "lengte",
                                           "bodemh_bov": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"].apply(lambda x: f"PRO_{x}")
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen duikers en siphons
duikers en sifons worden als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [9]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    snap_to_branches=snap_to_branches,
                                    attribute_filter={"SOORT": ["maaipadduiker",
                                                                "slootduiker",
                                                                "wegduiker",
                                                                "nog niet gevuld"]},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([culverts_gdf,siphons_gdf], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=5)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20

### Toevoegen stuwen

In [10]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=10)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [11]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={"gecom_max_": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [12]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [13]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]
con_gdf = hydamo.gemalen[~hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf = gpd.GeoDataFrame(data={"code": con_gdf["code"].values,
                                 "geometry": con_gdf["geometry"].values})
con_gdf["codegerelateerdobject"] = con_gdf["code"]
con_gdf["streefwaarde"] = -999

gdf = gpd.GeoDataFrame(pd.concat([gpg_gdf, con_gdf], ignore_index=True))

gdf["bovenmarge"] = gdf["streefwaarde"] + 0.05
gdf["ondermarge"] = gdf["streefwaarde"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

### Filteren

Alle branches die niet bij "Boezemmodel_v4" gooien we uit het model. Idem voor alle kunstwerken + profielen die naar deze branches zijn gesnapped. Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [14]:
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/boezemmodel_v4"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/boezemmodel_v4.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [15]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [16]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-02-25 22:32:09,541 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Aanmaken 1d netwerk

In [17]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-02-25 22:32:12,001 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-02-25 22:32:12,006 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000010 at: [0.0, 691.94, 1642.4933902054406], due to the structures at [-0.001, 671.552, 712.328, 1163.135, 1539.892, 1642.4943902054406].
2021-02-25 22:32:12,009 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000017 at: [0.0, 506.58549999999997, 522.864, 971.4883654003871], due to the structures at [-0.001, 25.63, 498.27, 514.901, 530.827, 971.4893654003871].
2021-02-25 22:32:12,021 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000118 at: [0.0, 6.069, 1220.3091726956434], due to the structures at [-0.001, 0.1, 12.038, 1220.3101726956434].
2021-02-25 22:32:12,026 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000198 at: [0.0, 662.9685, 687.5405, 712.5396643085704], due to the structures at [-0.001, 0.581, 363.777,

2021-02-25 22:32:12,183 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001387 at: [0.0, 6.8325, 623.2646597548901], due to the structures at [-0.001, 0.1, 13.565, 623.1646597548901, 623.2656597548901].
2021-02-25 22:32:12,187 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001460 at: [0.0, 237.6645, 289.8665, 369.35670479819925], due to the structures at [-0.001, 225.959, 249.37, 330.363, 369.3577047981992].
2021-02-25 22:32:12,190 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001493 at: [0.0, 827.081, 843.7500297007324], due to the structures at [-0.001, 292.981, 823.096, 831.066, 843.7510297007324].
2021-02-25 22:32:12,200 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001567 at: [0.0, 298.3915, 319.74350000000004, 355.1155, 398.4674689694366], due to the structures at [-0.001, 146.245, 295.019, 301.764, 337.723, 372.508, 398.46846896943657].
2021-02-25 22:32:12,203 - delft3dfmpy.core.dfm 

2021-02-25 22:32:12,331 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002683 at: [0.0, 316.0345, 322.79150000000004, 329.6585, 362.04449999999997, 929.9320316633184], due to the structures at [-0.001, 314.921, 317.148, 328.435, 330.882, 393.207, 929.9330316633184].
2021-02-25 22:32:12,338 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002728 at: [0.0, 10.980500000000001, 38.71202945346879], due to the structures at [-0.001, 0.1, 21.861, 38.713029453468785].
2021-02-25 22:32:12,343 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002820 at: [0.0, 1.9105, 592.6156206353999], due to the structures at [-0.001, 0.1, 3.721, 592.6166206353998].
2021-02-25 22:32:12,345 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002834 at: [0.0, 271.9015, 399.40599717622445], due to the structures at [-0.001, 38.474, 269.919, 273.884, 399.4069971762244].
2021-02-25 22:32:12,351 - delft3dfmpy.core.dfm - dfm - INFO 

2021-02-25 22:32:12,498 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004516 at: [0.0, 8.120000000000001, 24.117, 40.0375, 56.0055, 71.981, 87.906, 103.83099999999999, 119.7995, 135.875, 151.844, 193.59999214244925], due to the structures at [-0.001, 0.1, 16.14, 32.094, 47.981, 64.03, 79.932, 95.88, 111.782, 127.817, 143.933, 159.755, 193.60099214244926].
2021-02-25 22:32:12,500 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004520 at: [0.0, 12.807, 314.19658569060636], due to the structures at [-0.001, 12.279, 13.335, 314.19758569060633].
2021-02-25 22:32:12,502 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004525 at: [0.0, 254.77300000000002, 302.7180258495953], due to the structures at [-0.001, 247.388, 262.158, 302.71902584959525].
2021-02-25 22:32:12,505 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004527 at: [0.0, 234.53750000000002, 653.2525740465761], due to the structures at [-0

2021-02-25 22:32:12,655 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005696 at: [0.0, 25.454, 44.06007262813716], due to the structures at [-0.001, 17.027, 33.881, 44.06107262813716].
2021-02-25 22:32:12,657 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005704 at: [0.0, 27.941000000000003, 57.04910165814357], due to the structures at [-0.001, 1.831, 54.051, 57.05010165814357].
2021-02-25 22:32:12,662 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005812 at: [0.0, 624.1055, 636.3869427737824], due to the structures at [-0.001, 614.673, 633.538, 636.3879427737824].
2021-02-25 22:32:12,665 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005828 at: [0.0, 1519.4855, 1540.0675, 1556.4765, 1834.6190299779007], due to the structures at [-0.001, 1228.832, 1508.698, 1530.273, 1549.862, 1563.091, 1834.6200299779007].
2021-02-25 22:32:12,672 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on

2021-02-25 22:32:12,854 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016490 at: [0.0, 17.044, 25.366999999999997, 32.672, 40.668, 48.7805, 56.784, 64.7375, 76.902, 89.005, 97.0055, 104.9445, 112.88300000000001, 120.8965, 128.91899999999998, 163.10704631207375], due to the structures at [-0.001, 12.03, 22.058, 28.676, 36.668, 44.668, 52.893, 60.675, 68.8, 85.004, 93.006, 101.005, 108.884, 116.882, 124.911, 132.927, 163.10804631207375].
2021-02-25 22:32:12,856 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016498 at: [0.0, 26.388, 65.41386778463344], due to the structures at [-0.001, 0.1, 52.676, 65.41486778463344].
2021-02-25 22:32:12,859 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016504 at: [0.0, 971.2915, 1529.0315, 1805.6755, 2111.312359123833], due to the structures at [-0.001, 238.432, 684.778, 954.755, 987.828, 1528.719, 1529.344, 1800.862, 1810.489, 2111.3133591238334].
2021-02-25 22:32:12,861 - delft3d

### Toevoegen cross-sections

We kennen profielen toe, waar we profielen missen vullen we deze aan met principeprofielen.

In [18]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

2021-02-25 22:32:14,973 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 2385.
2021-02-25 22:32:24,053 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 863.
No parametrised crossections available for branches.
2021-02-25 22:32:24,061 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 636.
2021-02-25 22:32:24,090 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 636.
2021-02-25 22:32:24,143 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
863 branches are still missing a cross section.
0 structures are still missing a cross section.
adding trapezium profiles on branches with missing crosssections.


### Wegschrijven model

In [19]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\boezemmodel", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r"modellen\boezemmodel\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>